## Region init

In [1]:
import pandas as pd
import geopandas as gpd

towns = gpd.read_parquet('data/_towns.parquet')
settlements = gpd.read_parquet('data/_okrugs.parquet')
districts = gpd.read_parquet('data/_rayons.parquet')
adj_mx = pd.read_pickle('data/_adj_mx.pickle')

In [2]:
from townsnet.models.region import Region

region = Region(
  districts=districts, 
  settlements=settlements, 
  towns=towns,
  accessibility_matrix=adj_mx
)

In [3]:
region.get_towns_gdf()

,town_name,population,geometry,settlement_name,district_name
id,,,,,
0,Болото,111,POINT (543142.339 6580637.744),Борское сельское поселение,Бокситогорский муниципальный район
1,Большой Остров,116,POINT (544532.929 6593228.214),Борское сельское поселение,Бокситогорский муниципальный район
2,Бор,116,POINT (544895.402 6593052.876),Борское сельское поселение,Бокситогорский муниципальный район
3,Бороватое,114,POINT (543993.111 6589582.121),Борское сельское поселение,Бокситогорский муниципальный район
4,Бочево,108,POINT (538540.110 6576793.684),Борское сельское поселение,Бокситогорский муниципальный район
...,...,...,...,...,...
2926,Апраксин Бор,96,POINT (399059.742 6560341.471),Трубникоборское сельское поселение,Тоснненский муниципальный район
2927,Александровка,98,POINT (403967.171 6562087.081),Трубникоборское сельское поселение,Тоснненский муниципальный район
2928,Большая Горка,100,POINT (413116.740 6573793.279),Трубникоборское сельское поселение,Тоснненский муниципальный район


In [4]:
region.get_service_types_df()

,category,infrastructure,name,name_ru,weight,accessibility,demand
0,BASIC,EDUCATION,kindergarten,детский сад,0.20,7,61
1,BASIC,EDUCATION,school,школа,0.20,15,120
2,BASIC,HEALTHCARE,health_center,ФАП / амбулатория,0.20,10,13
3,BASIC,HEALTHCARE,pharmacy,аптека,0.20,15,15
4,BASIC,COMMERCE,convenience,продуктовый магазин,0.20,5,180
5,BASIC,COMMERCE,houseware,хозяйственный магазин,0.20,5,180
6,BASIC,CATERING,cafe,кафе / кофейня,0.15,15,25
7,BASIC,LEISURE,sports_hall,универсальный зал,0.10,15,180
8,BASIC,RECREATION,playground,детская площадка,0.20,4,2
9,BASIC,RECREATION,park,сквер / бульвар / лесопарк,0.20,30,150


## Fill service types data

In [5]:
import osmnx as ox

polygon = region.geometry
gdfs = {}
for service_type in region.service_types:
  try:
    gdf = gpd.read_parquet(f'data/services/{service_type.name}.parquet')
  except:
    gdf = ox.features_from_polygon(polygon, tags=service_type.osm_tags).reset_index().to_crs(region.crs)
    gdf = region.match_services_towns(gdf)
    gdf['capacity'] = 250
    gdf.geometry = gdf.representative_point()
  gdfs[service_type.name] = gdf

In [6]:
for service_type, gdf in gdfs.items():
  region.update_services(service_type, gdf)

In [7]:
region.get_services_gdf()

,town,service_type,geometry,capacity
0,Бор,kindergarten,POINT (545062.236 6592984.549),250
1,Бор,school,POINT (545003.453 6592946.638),250
2,Бор,cafe,POINT (545057.111 6593074.763),250
3,Дмитрово,bus_stop,POINT (556606.517 6570799.269),250
4,Ларьян,school,POINT (543946.962 6596292.051),250
...,...,...,...,...
20885,Апраксин Бор,bus_stop,POINT (399227.549 6560566.367),250
20886,Апраксин Бор,bus_stop,POINT (399216.723 6560632.806),250
20887,Большая Горка,bus_stop,POINT (413170.857 6573826.061),250
20888,Большая Горка,parking,POINT (413177.388 6573887.691),250


In [8]:
region.to_pickle('data/region.pickle')

## Provision calculate